In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsRegressor

import time


<h1>Links to all the resources I used to build this notebook:<h1>

Filling in missing values and value encoding:
https://www.kaggle.com/code/jillanisofttech/job-salary-prediction-by-jst

KKNR syntax and values:
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

hyper parameter tuning:
https://medium.datadriveninvestor.com/k-nearest-neighbors-in-python-hyperparameters-tuning-716734bc557f

After realizing Grid Search CV would take ages to run, found a faster grid search here:
https://towardsdatascience.com/11-times-faster-hyperparameter-tuning-with-halvinggridsearch-232ed0160155

In [2]:
#Get dataset
df = pd.read_csv("Train_rev1_2.csv") 
df.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


<h1>Cleaning and Encoding the whole dataset before splitting it up<h1>



In [3]:
#check missing values
df.isna().sum()

Id                         0
Title                      1
FullDescription            0
LocationRaw                0
LocationNormalized         0
ContractType          179326
ContractTime           63905
Company                32430
Category                   0
SalaryRaw                  0
SalaryNormalized           0
SourceName                 1
dtype: int64

In [4]:
#Turn all string values into category values
for label, content in df.items():
    if pd.api.types.is_string_dtype(content):
        df[label] = content.astype("category").cat.as_ordered()
    

In [5]:
#Filling in missing values and perform encoding
for label,content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
        # Add binary column to indicate whether sample had missing value
        df[label+"is_missing"]=pd.isnull(content)
        # Turn categories into numbers and add+1
        df[label] = pd.Categorical(content).codes+1
df.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,...,Titleis_missing,FullDescriptionis_missing,LocationRawis_missing,LocationNormalizedis_missing,ContractTypeis_missing,ContractTimeis_missing,Companyis_missing,Categoryis_missing,SalaryRawis_missing,SourceNameis_missing
0,12612628,39227,68301,5178,751,0,2,7758,9,21344,...,False,False,False,False,True,False,False,False,False,False
1,12612830,121388,196351,6881,987,0,2,7758,9,32427,...,False,False,False,False,True,False,False,False,False,False
2,12612844,73486,117824,7630,1069,0,2,7758,9,21539,...,False,False,False,False,True,False,False,False,False,False
3,12613049,39228,68300,17183,2310,0,2,7758,9,31840,...,False,False,False,False,True,False,False,False,False,False
4,12613647,85435,157839,17183,2310,0,2,7758,9,21344,...,False,False,False,False,True,False,False,False,False,False


In [6]:
#check for missing values again
df.isna().sum()

Id                              0
Title                           0
FullDescription                 0
LocationRaw                     0
LocationNormalized              0
ContractType                    0
ContractTime                    0
Company                         0
Category                        0
SalaryRaw                       0
SalaryNormalized                0
SourceName                      0
Titleis_missing                 0
FullDescriptionis_missing       0
LocationRawis_missing           0
LocationNormalizedis_missing    0
ContractTypeis_missing          0
ContractTimeis_missing          0
Companyis_missing               0
Categoryis_missing              0
SalaryRawis_missing             0
SourceNameis_missing            0
dtype: int64

In [7]:
#Train-validation-test split

X = df.drop(columns = ['SalaryNormalized']).copy()
y = df['SalaryNormalized']
# Split the data in training and remaining dataset, train size = 60%
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.6)
# Split remaining 50/50 into validation and test sets
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(146860, 21)
(146860,)
(48954, 21)
(48954,)
(48954, 21)
(48954,)


(None, None)

In [8]:
#Initialize basic KNN regression model:
KNNR = KNeighborsRegressor(n_neighbors=10)


In [9]:
#Fit model to training data
KNNR.fit(X_train,y_train)

KNeighborsRegressor(n_neighbors=10)

In [10]:
#make prediction on validation data
y_pred_valid = KNNR.predict(X_valid)

In [11]:
#analyze results of prediction
MAE = mean_absolute_error(y_valid,y_pred_valid)

r2 = r2_score(y_valid,y_pred_valid)
print(f'MAE validation: {MAE}\nr2 validation: {r2}')
print('\n')


MAE validation: 9913.842137516853
r2 validation: 0.27429773090495924




In [12]:
#If you're curious what the individual predictions look like
n=0
for value in y_valid:
    print(f'Predicted: {y_pred_valid[n]} --> Actual: {value} ')
    n+=1

Predicted: 29180.0 --> Actual: 27500 
Predicted: 21310.0 --> Actual: 17500 
Predicted: 25203.2 --> Actual: 42240 
Predicted: 70275.0 --> Actual: 11884 
Predicted: 32846.8 --> Actual: 43000 
Predicted: 20700.0 --> Actual: 17000 
Predicted: 45340.0 --> Actual: 55000 
Predicted: 42850.0 --> Actual: 39500 
Predicted: 26200.0 --> Actual: 24750 
Predicted: 44568.0 --> Actual: 65000 
Predicted: 17543.6 --> Actual: 19000 
Predicted: 19705.0 --> Actual: 31800 
Predicted: 24735.9 --> Actual: 18500 
Predicted: 41586.0 --> Actual: 45000 
Predicted: 47600.0 --> Actual: 35000 
Predicted: 32230.0 --> Actual: 15600 
Predicted: 27042.9 --> Actual: 24000 
Predicted: 17325.0 --> Actual: 13440 
Predicted: 20855.5 --> Actual: 17250 
Predicted: 41330.0 --> Actual: 37500 
Predicted: 25444.0 --> Actual: 22318 
Predicted: 20278.8 --> Actual: 16000 
Predicted: 24150.0 --> Actual: 23000 
Predicted: 40950.0 --> Actual: 42500 
Predicted: 18051.2 --> Actual: 15500 
Predicted: 20827.0 --> Actual: 15500 
Predicted: 2

Predicted: 33900.0 --> Actual: 32500 
Predicted: 36578.2 --> Actual: 23000 
Predicted: 43576.0 --> Actual: 47500 
Predicted: 25016.0 --> Actual: 18250 
Predicted: 32325.0 --> Actual: 19500 
Predicted: 35419.3 --> Actual: 44160 
Predicted: 20532.5 --> Actual: 16000 
Predicted: 20470.3 --> Actual: 21500 
Predicted: 23633.8 --> Actual: 15000 
Predicted: 25804.8 --> Actual: 25000 
Predicted: 21355.3 --> Actual: 30720 
Predicted: 38125.0 --> Actual: 47500 
Predicted: 26950.0 --> Actual: 27500 
Predicted: 29559.2 --> Actual: 29958 
Predicted: 35250.0 --> Actual: 32000 
Predicted: 55778.0 --> Actual: 65000 
Predicted: 54300.0 --> Actual: 42500 
Predicted: 19309.8 --> Actual: 26880 
Predicted: 30725.0 --> Actual: 37500 
Predicted: 27040.7 --> Actual: 35000 
Predicted: 38450.0 --> Actual: 33000 
Predicted: 35282.0 --> Actual: 30794 
Predicted: 34826.0 --> Actual: 25000 
Predicted: 21250.0 --> Actual: 20500 
Predicted: 38900.0 --> Actual: 29500 
Predicted: 47200.0 --> Actual: 55000 
Predicted: 4

Predicted: 56692.5 --> Actual: 50000 
Predicted: 59020.0 --> Actual: 42500 
Predicted: 23100.0 --> Actual: 25000 
Predicted: 39110.0 --> Actual: 35000 
Predicted: 43650.0 --> Actual: 37500 
Predicted: 38856.1 --> Actual: 51500 
Predicted: 34187.2 --> Actual: 33000 
Predicted: 29880.0 --> Actual: 26600 
Predicted: 41800.0 --> Actual: 37500 
Predicted: 28142.5 --> Actual: 27500 
Predicted: 31371.6 --> Actual: 29098 
Predicted: 60360.4 --> Actual: 50000 
Predicted: 42700.0 --> Actual: 17500 
Predicted: 29227.8 --> Actual: 30000 
Predicted: 62200.0 --> Actual: 25800 
Predicted: 35100.4 --> Actual: 25000 
Predicted: 54900.0 --> Actual: 80000 
Predicted: 38100.0 --> Actual: 32000 
Predicted: 45410.0 --> Actual: 22080 
Predicted: 26394.4 --> Actual: 26000 
Predicted: 35862.1 --> Actual: 26000 
Predicted: 25600.0 --> Actual: 19000 
Predicted: 52450.0 --> Actual: 50000 
Predicted: 34648.0 --> Actual: 18750 
Predicted: 34841.3 --> Actual: 32500 
Predicted: 28875.0 --> Actual: 20500 
Predicted: 5

Predicted: 28300.0 --> Actual: 27500 
Predicted: 27850.0 --> Actual: 17700 
Predicted: 35457.6 --> Actual: 42500 
Predicted: 22409.4 --> Actual: 18000 
Predicted: 33405.3 --> Actual: 25000 
Predicted: 25543.0 --> Actual: 33600 
Predicted: 37900.0 --> Actual: 40000 
Predicted: 34400.0 --> Actual: 59520 
Predicted: 46083.4 --> Actual: 31500 
Predicted: 37250.0 --> Actual: 35000 
Predicted: 51134.3 --> Actual: 52500 
Predicted: 33239.3 --> Actual: 18000 
Predicted: 32127.8 --> Actual: 30000 
Predicted: 37200.0 --> Actual: 40000 
Predicted: 53750.0 --> Actual: 50000 
Predicted: 36774.0 --> Actual: 30000 
Predicted: 18293.1 --> Actual: 15000 
Predicted: 21475.0 --> Actual: 20500 
Predicted: 35950.0 --> Actual: 40000 
Predicted: 32916.0 --> Actual: 24500 
Predicted: 29957.7 --> Actual: 26500 
Predicted: 25400.0 --> Actual: 21000 
Predicted: 38230.0 --> Actual: 52500 
Predicted: 29700.0 --> Actual: 84000 
Predicted: 27682.0 --> Actual: 15000 
Predicted: 28375.0 --> Actual: 72000 
Predicted: 3

Predicted: 35100.0 --> Actual: 27500 
Predicted: 14856.8 --> Actual: 11884 
Predicted: 47950.0 --> Actual: 41000 
Predicted: 27440.0 --> Actual: 10320 
Predicted: 35700.0 --> Actual: 36500 
Predicted: 44214.0 --> Actual: 26400 
Predicted: 26756.2 --> Actual: 11884 
Predicted: 39746.0 --> Actual: 32400 
Predicted: 43876.0 --> Actual: 70000 
Predicted: 43160.0 --> Actual: 35000 
Predicted: 29342.5 --> Actual: 23500 
Predicted: 27077.1 --> Actual: 29400 
Predicted: 19185.0 --> Actual: 19500 
Predicted: 27030.0 --> Actual: 35000 
Predicted: 35102.0 --> Actual: 27000 
Predicted: 25315.0 --> Actual: 17250 
Predicted: 24938.0 --> Actual: 21120 
Predicted: 54053.0 --> Actual: 50000 
Predicted: 46527.8 --> Actual: 75000 
Predicted: 28700.0 --> Actual: 28500 
Predicted: 45900.0 --> Actual: 45000 
Predicted: 50390.0 --> Actual: 24000 
Predicted: 40999.8 --> Actual: 40000 
Predicted: 22183.1 --> Actual: 18000 
Predicted: 49915.0 --> Actual: 62500 
Predicted: 44935.2 --> Actual: 37500 
Predicted: 4

...

Predicted: 31938.0 --> Actual: 22500 
Predicted: 19118.6 --> Actual: 16000 
Predicted: 36986.0 --> Actual: 42500 
Predicted: 46050.0 --> Actual: 65000 
Predicted: 27347.6 --> Actual: 15436 
Predicted: 31665.0 --> Actual: 27500 
Predicted: 47196.0 --> Actual: 50000 
Predicted: 47481.7 --> Actual: 42500 
Predicted: 19526.2 --> Actual: 19000 
Predicted: 34856.9 --> Actual: 40000 
Predicted: 37677.5 --> Actual: 28000 
Predicted: 40359.5 --> Actual: 52500 
Predicted: 48844.0 --> Actual: 40000 
Predicted: 67599.8 --> Actual: 26500 
Predicted: 36460.0 --> Actual: 42000 
Predicted: 29568.0 --> Actual: 18800 
Predicted: 21872.0 --> Actual: 15000 
Predicted: 51539.9 --> Actual: 62499 
Predicted: 20810.0 --> Actual: 15500 
Predicted: 21796.6 --> Actual: 21500 
Predicted: 45150.0 --> Actual: 52500 
Predicted: 52500.0 --> Actual: 69000 
Predicted: 39379.0 --> Actual: 36500 
Predicted: 40030.0 --> Actual: 21000 
Predicted: 28090.0 --> Actual: 47500 
Predicted: 23652.4 --> Actual: 70000 
Predicted: 3

Predicted: 27519.3 --> Actual: 27000 
Predicted: 36075.0 --> Actual: 26000 
Predicted: 31742.3 --> Actual: 36864 
Predicted: 29536.1 --> Actual: 59520 
Predicted: 33516.0 --> Actual: 28000 
Predicted: 39960.0 --> Actual: 37500 
Predicted: 20090.6 --> Actual: 36480 
Predicted: 22838.0 --> Actual: 32400 
Predicted: 46751.0 --> Actual: 37500 
Predicted: 14452.6 --> Actual: 17875 
Predicted: 22610.6 --> Actual: 19500 
Predicted: 25200.0 --> Actual: 25000 
Predicted: 23429.0 --> Actual: 21500 
Predicted: 33720.7 --> Actual: 15552 
Predicted: 37950.0 --> Actual: 40000 
Predicted: 20768.0 --> Actual: 15000 
Predicted: 13014.3 --> Actual: 11884 
Predicted: 50850.0 --> Actual: 40000 
Predicted: 40455.5 --> Actual: 13843 
Predicted: 42368.0 --> Actual: 39000 
Predicted: 19732.7 --> Actual: 20500 
Predicted: 35146.0 --> Actual: 29500 
Predicted: 44567.5 --> Actual: 80000 
Predicted: 26214.8 --> Actual: 32640 
Predicted: 50064.0 --> Actual: 90000 
Predicted: 28340.0 --> Actual: 22107 
Predicted: 5

Predicted: 20075.0 --> Actual: 20000 
Predicted: 19511.0 --> Actual: 21000 
Predicted: 32300.0 --> Actual: 33500 
Predicted: 22813.1 --> Actual: 37200 
Predicted: 60869.8 --> Actual: 90000 
Predicted: 15704.4 --> Actual: 28252 
Predicted: 46002.8 --> Actual: 31000 
Predicted: 28624.0 --> Actual: 21500 
Predicted: 30077.5 --> Actual: 20400 
Predicted: 17040.4 --> Actual: 14404 
Predicted: 24400.0 --> Actual: 29750 
Predicted: 41866.0 --> Actual: 22080 
Predicted: 32226.0 --> Actual: 26500 
Predicted: 22703.6 --> Actual: 22000 
Predicted: 55915.0 --> Actual: 20160 
Predicted: 21690.7 --> Actual: 24336 
Predicted: 23840.0 --> Actual: 24000 
Predicted: 17344.4 --> Actual: 21312 
Predicted: 33008.4 --> Actual: 20000 
Predicted: 28518.0 --> Actual: 24000 
Predicted: 27192.0 --> Actual: 52500 
Predicted: 20083.3 --> Actual: 10500 
Predicted: 45870.0 --> Actual: 15000 
Predicted: 34750.0 --> Actual: 40000 
Predicted: 23452.8 --> Actual: 32500 
Predicted: 43860.0 --> Actual: 32500 
Predicted: 4

Predicted: 30301.1 --> Actual: 18143 
Predicted: 52622.5 --> Actual: 14400 
Predicted: 34154.4 --> Actual: 55000 
Predicted: 31704.0 --> Actual: 32500 
Predicted: 56228.0 --> Actual: 75000 
Predicted: 34549.9 --> Actual: 90000 
Predicted: 40400.0 --> Actual: 35000 
Predicted: 25350.0 --> Actual: 27500 
Predicted: 31850.9 --> Actual: 53581 
Predicted: 41162.0 --> Actual: 41000 
Predicted: 28752.0 --> Actual: 26000 
Predicted: 49021.9 --> Actual: 13516 
Predicted: 34764.0 --> Actual: 40000 
Predicted: 44335.0 --> Actual: 87000 
Predicted: 27728.0 --> Actual: 23500 
Predicted: 36237.0 --> Actual: 10000 
Predicted: 50116.4 --> Actual: 80000 
Predicted: 26296.0 --> Actual: 40320 
Predicted: 34613.7 --> Actual: 27750 
Predicted: 30750.0 --> Actual: 12500 
Predicted: 27002.6 --> Actual: 27000 
Predicted: 18893.3 --> Actual: 14100 
Predicted: 27522.4 --> Actual: 32750 
Predicted: 48079.8 --> Actual: 14515 
Predicted: 51850.0 --> Actual: 38000 
Predicted: 34325.0 --> Actual: 30000 
Predicted: 3

Predicted: 16995.8 --> Actual: 18240 
Predicted: 45725.4 --> Actual: 150000 
Predicted: 21218.0 --> Actual: 19796 
Predicted: 27351.6 --> Actual: 10000 
Predicted: 24519.0 --> Actual: 16800 
Predicted: 30248.4 --> Actual: 20800 
Predicted: 34910.0 --> Actual: 26500 
Predicted: 43050.0 --> Actual: 42500 
Predicted: 41000.0 --> Actual: 37500 
Predicted: 23620.0 --> Actual: 15000 
Predicted: 25400.1 --> Actual: 13440 
Predicted: 24086.4 --> Actual: 30720 
Predicted: 22839.4 --> Actual: 16080 
Predicted: 26735.0 --> Actual: 24000 
Predicted: 27550.0 --> Actual: 24000 
Predicted: 24875.0 --> Actual: 31500 
Predicted: 26208.0 --> Actual: 22000 
Predicted: 39585.0 --> Actual: 42500 
Predicted: 30028.0 --> Actual: 28500 
Predicted: 26877.4 --> Actual: 13440 
Predicted: 21226.9 --> Actual: 23923 
Predicted: 25625.0 --> Actual: 24000 
Predicted: 34346.1 --> Actual: 138240 
Predicted: 43324.0 --> Actual: 36000 
Predicted: 41550.0 --> Actual: 37500 
Predicted: 41034.4 --> Actual: 30000 
Predicted:

In [14]:
#Let op! This took ~51 minutes to run on my 2015 macbook
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

#Hyperparameter tuning, these are the params we wanna tune, understand what they all mean for the purpose of
#writing the report
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
KNNR_2 = KNeighborsRegressor()


#Here a Halving Grid Search CV is used to find the best hyperparameters for the model
#It takes a long time but is much faster than classic Grid Search CV, at roughly equivalent performance
halving_cv = HalvingGridSearchCV(
    KNNR_2, hyperparameters, n_jobs=-1, cv=3, min_resources="exhaust", factor=3,verbose = 10)


print("HalvingGrid search...")
search_time_start = time.time()
best_model = halving_cv.fit(X_train,y_train)
print("Grid search time:", time.time() - search_time_start)
print('\n')

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

HalvingGrid search...
n_iterations: 8
n_required_iterations: 8
n_possible_iterations: 8
min_resources_: 67
max_resources_: 146860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 2842
n_resources: 67
Fitting 3 folds for each of 2842 candidates, totalling 8526 fits
----------
iter: 1
n_candidates: 948
n_resources: 201
Fitting 3 folds for each of 948 candidates, totalling 2844 fits
----------
iter: 2
n_candidates: 316
n_resources: 603
Fitting 3 folds for each of 316 candidates, totalling 948 fits
----------
iter: 3
n_candidates: 106
n_resources: 1809
Fitting 3 folds for each of 106 candidates, totalling 318 fits
----------
iter: 4
n_candidates: 36
n_resources: 5427
Fitting 3 folds for each of 36 candidates, totalling 108 fits


/Users/jerenolsen/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


----------
iter: 5
n_candidates: 12
n_resources: 16281
Fitting 3 folds for each of 12 candidates, totalling 36 fits
----------
iter: 6
n_candidates: 4
n_resources: 48843
Fitting 3 folds for each of 4 candidates, totalling 12 fits
----------
iter: 7
n_candidates: 2
n_resources: 146529
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Grid search time: 3085.4615092277527
None


Best leaf_size: 9
Best p: 1
Best n_neighbors: 24


In [22]:
#Train new model with best hyper parameters
#KNNR_3 = KNeighborsRegressor(leaf_size = 9,p=1,n_neighbors = 24)
#tuned_model = KNNR_3.fit(X_train,y_train)
y_pred_tuned = best_model.predict(X_valid)

In [23]:
#Evaluate new model, compare its performance to the first
MAE_tuned = mean_absolute_error(y_valid,y_pred_tuned)

r2_tuned = r2_score(y_valid,y_pred_tuned)
print(f'MAE of tuned model: {MAE_tuned}\nr2 tuned model: {r2_tuned}')
print('\n')
print(f'MAE before hyper param tuning:{MAE}\nr2 before tuning: {r2}')

MAE of tuned model: 9958.36160136727
r2 tuned model: 0.2993190618316982


MAE before hyper param tuning:9913.842137516853
r2 before tuning: 0.27429773090495924


In [ ]:
#Turns out hyper param tuning in this way didn't really improve our model... 
#Might want to look around for other approaches

In [ ]:
#Finally, run the model on the test data to get test predictions, final model evaluation

# This is hyper param tuning with classic Grid Search CV
#This step takes 6 hours and might light your computer on fire

#Hyperparameter tuning, these are the params we wanna tune, understand what they all mean for the purpose of
#writing the report
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

#Create new KNN object
KNNR_2 = KNeighborsRegressor()


#Use GridSearch, understand how this works for writing report
clf = GridSearchCV(KNNR_2, hyperparameters, cv=3, n_jobs = -1, verbose = 10)

#Fit the model

print("Grid search...")
search_time_start = time.time()

best_model = clf.fit(X_train,y_train)
print(print("Grid search time:", time.time() - search_time_start))
print('\n')

#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

